## Contexto de negocio 
Es analista de datos en una gran empresa de servicios financieros que vende una cartera diversa de productos. Para realizar estas ventas, la firma cuenta con un centro de llamadas donde los agentes de ventas hacen llamadas tanto a los clientes actuales como a los potenciales. A la empresa le gustaría que se sumergiera en sus datos para diseñar estrategias para aumentar sus ingresos o reducir sus costos. Específicamente, les gustaría duplicar sus clientes más confiables y eliminar a los agentes de ventas que no están produciendo resultados.

## Problema de negocio. 
La empresa desea responder a las siguientes preguntas: "¿Qué tipo de clientes son más probable que compren nuestro producto? ¿Y cuáles de mis agentes de ventas son los más/menos productivos?

In [6]:
"""
Instalar todo lo necesarios como:
sqlalchemy
engine sql 
pandas 
"""
%pip install -q sqlalchemy
%pip show sqlalchemy

import os 
os.getcwd()

import pandas as pd 
from  sqlalchemy import create_engine, text 
pd.options.display.max_rows = 100
ruta="./"




Note: you may need to restart the kernel to use updated packages.
Name: SQLAlchemy
Version: 2.0.29
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: /usr/local/python/3.10.13/lib/python3.10/site-packages
Requires: greenlet, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Preparando el espacio de trabajo

In [56]:
"""
Crear el motor sql para el manejo de la base de datos 
#engine = "sqlite:///mydatabase.db" # /// apunta hacia una ruta especifica
#engine=create_engine('sqlite://david_database.db') #indica que se está utilizando el motor de base de datos SQLite.
"""
engine = create_engine('sqlite://')  ##indica que se está utilizando el motor de base de datos SQLite."
print(engine)

"""
Aqui vamos a traer todas las tablas datos que necesitamos para enviarlas al SQL , la manda a una conection de base de datos que tengas abierta,
pide el nombre que quieres que tenga, el moptor al cual la enias y el index es solo para ver si quieres el num de las filas a la izquierda del cuadro general 
"""
df = pd.read_csv('customers.csv').to_sql('customer', engine, if_exists='replace', index=False)
df = pd.read_csv('agents.csv').to_sql('agent', engine, if_exists='replace', index=False)
df = pd.read_csv('calls.csv').to_sql('call', engine, if_exists='replace', index=False)

"""
Usamos el def run query(SQL)
para poder conectar todas las bases de datos que tenemos ademas de con fetchall agarramos todas las filas disponibles , luedo le asignamos las columns 
"""

def runQuery(sql):
    result = engine.connect().execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

Engine(sqlite://)


# Hello SQL con python

In [8]:
query_test = """ 
SELECT Name
FROM agent
Where Name LIKE '%A%'

"""
runQuery(query_test)



,name
0,Michele Williams
1,Jocelyn Parker
2,Randy Moore
3,Paul Nunez
4,Gloria Singh
5,Angel Briggs
6,Lisa Cordova
7,Dana Hardy
8,Agent X


# Ejercicio 1 

Escriba una consulta que seleccione el ID y el nombre del cliente de la tabla Customer, mostrando solo los resultados de los clientes que no están desempleados. Recuerde escribir su consulta como una cadena de varias líneas (encerrada entre un par de comillas triples """) y pasarla a la función runQuery()definida en el marco anterior para verificar su trabajo.

Respuesta.

In [46]:
query_EJ_1 = """
SELECT Cu.CustomerID, Cu.Name, Cu.Occupation
FROM Customer as Cu
WHERE Occupation != 'Unemployed'
ORDER BY Cu.name ASC


"""
runQuery(query_EJ_1)





,customerid,name,occupation
0,900,Aaron Gutierrez,IT consultant
1,622,Aaron Rose,"Engineer, production"
2,226,Adam Ward,Police officer
3,786,Alan Chambers,Administrator
4,985,Alan Mitchell,"Engineer, electrical"
...,...,...,...
755,699,Willie Greene,"Engineer, electronics"
756,715,Yesenia Wright,Orthoptist
757,952,Yolanda White,Chemical engineer
758,421,Zachary Ruiz,Chemical engineer


# Ejercicio 2 
Escriba una consulta que produzca una lista, en orden alfabético, de todas las distintas ocupaciones en la tabla Customer que contengan la palabra" Ingeniero ".

In [101]:
query_EJ_2= """
SELECT Cu.Name as Name ,Cu.Occupation as Occupation , Cu.PhoneNumber as Phone
FROM Customer as Cu
WHERE Occupation  LIKE '%engineer%'
ORDER BY Name ASC
"""
runQuery(query_EJ_2)

,Name,Occupation,Phone
0,Aaron Rose,"Engineer, production",192-727-2376
1,Alan Mitchell,"Engineer, electrical",454-752-1489
2,Alexis Riddle,"Engineer, mining",838-625-2101
3,Alice Lee,"Engineer, civil (consulting)",225-768-5812
4,Alison Vaughan,"Engineer, water",470-488-8589
...,...,...,...
356,William Garcia,"Engineer, broadcasting (operations)",483-863-4105
357,William Jackson,"Engineer, communications",939-210-6856
358,Willie Greene,"Engineer, electronics",366-748-8436
359,Yolanda White,Chemical engineer,706-755-6105


# Ejercicio 3 
Escriba una consulta que devuelva el ID del cliente, su nombre y una columna Over30 que contenga" Sí "si el cliente tiene más de 30 años y" No "si no.

Sugerencia: Deberá utilizar la cláusula CASE-END. La cláusula CASE-END se puede utilizar para evaluar declaraciones condicionales y devuelve un valor una vez que se cumple una condición (similar a una cláusula if-then-else en Python). Si no se cumple ninguna condición, devuelve el valor de la cláusula ELSE (o NULL si no hay una declaración ELSE). Por ejemplo:
CASE
    WHEN Name = "Matt" THEN 'Yes'
    WHEN Name = "Matteo" THEN 'Maybe'
    ELSE 'No'
END

In [136]:
query_EJ_3 = """
SELECT Cu.Name AS Name,Cu.CustomerID,Cu.AGE AS EDAD,Cu.Occupation,
  CASE
    WHEN AGE < 0 THEN 'WRONG DATA'
    WHEN AGE >= 30 THEN 'YES'
    WHEN AGE < 30 THEN 'NO'
    ELSE 'MISSING DATA'
  END AS 'Over 30'
FROM Customer as Cu
WHERE Occupation LIKE '%Engineer%' AND AGE IS NOT NULL 
ORDER BY EDAD DESC
LIMIT 10

"""

""" 
Aqui lo que hacemos es volver a pasarlo a df con pandas y filtrar esa info de  wrong data pero no es 100 % necesario ya que se puede hacer directamente con SQL
"""
df_EJ_3=runQuery(query_EJ_3)
df_fixed = df_EJ_3[df['Over 30']!='WRONG DATA']
df_EJ_3



/tmp/ipykernel_2507/1650946592.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_fixed = df_EJ_3[df['Over 30']!='WRONG DATA']


,Name,customerid,EDAD,occupation,Over 30
0,Jeremy Wheeler,848,58,"Engineer, maintenance (IT)",YES
1,Monica Krueger,190,49,Electrical engineer,YES
2,Kevin Flores,256,48,"Engineer, civil (contracting)",YES
3,Ebony Sanchez,761,48,"Engineer, materials",YES
4,Fred Weber MD,947,48,"Engineer, manufacturing systems",YES
5,Shannon Gomez,965,48,"Engineer, mining",YES
6,Shelby Rodriguez,596,47,Chemical engineer,YES
7,Heather Johnson,771,47,Chemical engineer,YES
8,Scott Norris,116,46,Chemical engineer,YES
9,Maria Young,445,46,"Engineer, structural",YES


# PART 2 JOINS 